## Генератор прикольчиков и раздаватель смешков

### Постановка задачи:
Обучим генеративную модель для генерации анекдотов

##### Сбор данных:
1. Скачаем базовый [датасет](https://disk.yandex.com/d/fjt5xICH-ukEEA) из открытых источников
2. Напишем парсер телеграмм каналов (telegram_parser.py), соберем еще анекдотов
3. Обработаем собранные анекдоты и расширим ими датасет

##### Построение модели:
1. Токенизируем тексты (пробуем sentencepiece bpe tokenizer с разным размером словаря)
2. Используем RNN и LSTM бейзлайн, обучая эмбеддинги вместе с моделью
3. С нуля обучаем декодер из Transformer (не сделано, но начало положено в папке Transformer. Там я хотел сначала протестировать его на классификации текстов)
4. Дообучаем BERT, GPT (не сделано)

##### Создание интерфейса:
1. Используем Gradio
2. Переезжаем на FastApi бэкенд (не сделано)


In [2]:
import torch
import os
import numpy as np
from torch.utils.data import DataLoader
from dataset import TextDataset

Пользуемся библиотекой [sentencepiece](https://github.com/google/sentencepiece) для создания датасета

In [4]:
EMBEDDING_DIM = 256
VOCAB_SIZE = 6000
RNN_LAYERS = 3
DROPOUT = 0.4

In [5]:
train_set = TextDataset(data_file='mega_jokes_dataset.txt', train=True, sp_model_prefix='bpe', vocab_size=VOCAB_SIZE)
valid_set = TextDataset(data_file='mega_jokes_dataset.txt', train=False, sp_model_prefix='bpe', vocab_size=VOCAB_SIZE)

In [6]:
# тесты
for _ in range(5):
    for dataset in (train_set, valid_set):
        indices, length = dataset[np.random.randint(len(dataset))]
        assert indices.shape == (dataset.max_length, )
        assert indices[0].item() == dataset.bos_id

        eos_pos = indices.tolist().index(dataset.eos_id)
        assert torch.all(indices[eos_pos + 1:] == dataset.pad_id)
        assert (indices != dataset.pad_id).sum() == length

#### Обучение бейзлайна
Архитектура: несколько слоев RNN / LSTM подряд

Гиперпараметры токенизатора:
    - Тип токенизации
    - Размер словаря

Гиперпараметры модели:
    - Размерность эмбеддингов
    - RNN / LSTM
    - Число слоев
    - Dropout между слоями

Гиперпараметры обучения:
    - Оптимизатор
    - Размер батча
    - Число эпох

In [7]:
from model import LanguageModel

In [8]:
model = LanguageModel(train_set, embed_size=EMBEDDING_DIM, hidden_size=EMBEDDING_DIM, rnn_layers=RNN_LAYERS, dropout=DROPOUT)

In [9]:
# тесты
for bs in [1, 4, 16, 64, 256]:
    indices = torch.randint(high=train_set.vocab_size, size=(bs, train_set.max_length))
    lengths = torch.randint(low=1, high=train_set.max_length + 1, size=(bs, ))
    logits = model(indices, lengths)
    assert logits.shape == (bs, lengths.max(), train_set.vocab_size)

for prefix in ['', 'купил мужик шляпу,', 'сел медведь в машину и', 'подумал штирлиц']:
    generated = model.inference(prefix, temp=np.random.uniform(0.1, 10))
    assert type(generated) == str
    assert generated.startswith(prefix)

In [12]:
import train

BATCH_SIZE = 256
NUM_EPOCHS = 7
MODEL_NAME = "baseline2"
MODEL_PATH = MODEL_NAME + "_checkpoint" + ".pt"

optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.93)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=True)

if not os.path.isfile(MODEL_PATH):
    train.train(model, optimizer, scheduler, train_loader, val_loader, NUM_EPOCHS, saving_path=MODEL_PATH)
else:
    state_dict = torch.load(MODEL_PATH)
    model.load_state_dict(state_dict["model_state_dict"])
    train.generate_examples(model)

Generation examples:
1. говорит мать людям весей. после чего уже учапели «сею, может это гаика? —поллекп? - нет, видно, здесь нет. у нас нет.
2. в юбку мужику и говорит: - девушка, купи мне спрабые кристи за добром
3. ская женщина делите -я у мепыле и олом до своих базараю наковалались пропасть "скократисты купить успех в полицию...- страшная?- да понюшал первымносата дорожном гестом. ну, что ты косяете?- оказывается, что ты такой юж: — вопрос машья? — а че, у нас смогу барабан? - или же тебе определить? она позавчера: идут недопрохиво, ничего и думал, что резятником. на голове и вдруг спрашивает мужу: в дом.
4. нас умерра?- как ты об этом начинаем?
5. девушки, где замечается ему что произошло? удивился как мы.


А теперь LSTM

In [15]:
BATCH_SIZE = 256
NUM_EPOCHS = 7
RNN_LAYERS = 2
MODEL_NAME = "lstm"
MODEL_PATH = MODEL_NAME + "_checkpoint" + ".pt"


model = LanguageModel(train_set, embed_size=EMBEDDING_DIM, hidden_size=EMBEDDING_DIM, rnn_layers=RNN_LAYERS, dropout=DROPOUT, rnn_type=torch.nn.LSTM)
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.93)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=True)

if not os.path.isfile(MODEL_PATH):
    train.train(model, optimizer, scheduler, train_loader, val_loader, NUM_EPOCHS, saving_path=MODEL_PATH)
else:
    state_dict = torch.load(MODEL_PATH)
    model.load_state_dict(state_dict["model_state_dict"])
    train.generate_examples(model)

Generation examples:
1. ребёнка шлаку и мордному сезтальтор дрюйца бупает в туффуку. съел секретаршил нафиг, третий день все равносплюлиста. иположенная люди в окоси один стоящий, только во всем его начинают ножаться выбраться, вдруг стоит в магазине каннялись, а написать раздается собесекан. но видит на третий 3 жизни саженчики объясняет вылезает класс. стал стыдника, ему полилой учления! ну утром теперь лешка, сейчас расставил коробку, и наблюскается и казь и спросил минуту поверляет,.
2. , умер, оглядьться. одна не знает, как асфальт,ва, она возмуспеваливают, наэй тишина, выпивается от глаза и говорят: - "не белов, негр польмер развратки и димёп и всё время?? ничё) подписаться в встресло купрое вопркой. состарился втроющий правоводелся. так, следущегое предпанным и чисту: - обиверной тебя (премнего увидел же сигарет. тот ловит курица унесенький и говорит: - привет, двач, как хуй наш заяц. второй: -
3. путарь притдортовов. выступает, на квадратных слабоктовы сидит поибориенными фазен

#### Предварительные выводы:
у бейзлайна сносит крышу, если честно.
Анекдоты, конечно, получились смешные, но далеко не потому что я построил хорошую модель...
У сгенерированных примеров видим слабую языковую структуру и очень много несуществующих слов, однако что-то модель все же выучила, и, читая примеры, мы можем даже догадаться, что это должны быть анекдоты)


#### Идеи по улучшению:
1. Я допустил багу, ее нао найти и пофиксить)
2. Происходят взрывы градиентов, стоит попробовать клиппинг в цикле обучения
3. Выучить эмбеддинги отдельно от модели (Например, использовать word2vec из Embeddings.ipynb)
4. У модели около 4М параметров, так что возможно набор данных для нее очень мал, нужно использовать fine-tuning, чтобы сначала выучить языковую структуру